In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
shared_idx, shared_col, collective_frames = {}, {}, {}
for root, dirs, files in os.walk("../third-party/Greenhouse Gas"):
    for filename in files:
        if filename[-4:] == ".dat":
            # Transpose to create a multivariate view of the problem
            dataframe = pd.read_csv("%s/%s" % (root, filename), sep=' ', header=None, index_col=None).T
            # Identifying the shared timestamps and shared columns
            shared_idx = set(dataframe.index) if len(shared_idx) == 0 else shared_idx & set(dataframe.index)
            shared_col = set(dataframe.columns) if len(shared_col) == 0 else shared_col & set(dataframe.columns)
            # Saving the current dataset for later use
            collective_frames[filename.split(".")[2]] = dataframe

In [3]:
shared_idx = sorted(shared_idx)
shared_col = sorted(shared_col)

In [4]:
dataframes = []
for key in list(sorted(collective_frames.keys())):
    dataframe = collective_frames[key].loc[shared_idx]
    dataframe = dataframe[shared_col]
    dataframes.append(dataframe.values)
matrix = np.stack(dataframes)

In [5]:
shared_idx = [str(idx) for idx in shared_idx]
shared_col = [str(idx) for idx in shared_col]

In [6]:
np.save("../prepared-data/Greenhouse Gas (%dx%dx%d)" % matrix.shape, matrix)

In [7]:
with open("../prepared-data/Greenhouse Gas (%dx%dx%d).AX0" % matrix.shape,'w') as f:
    f.write('\n'.join(list(sorted(collective_frames.keys()))))

In [8]:
with open("../prepared-data/Greenhouse Gas (%dx%dx%d).AX1" % matrix.shape,'w') as f:
    f.write('\n'.join(shared_idx))

In [9]:
with open("../prepared-data/Greenhouse Gas (%dx%dx%d).AX2" % matrix.shape,'w') as f:
    f.write('\n'.join(shared_col))